<!-- primer.md – Theory Primer for Static Discrete Games -->
<!-- Render with Marp, Remark, or any markdown‑to‑slides tool               -->

# Theory Primer  
*Empirical IO Discrete‑Game Identification Toolkit*

Ten concise points that give you the minimum theory needed to run the code.

---

## 1 The Setup

\begin{aligned}
&\text{Players } j = 1,\dots,n,\qquad a_j\in\{0,1\} \\[2pt]
&z_j \text{ : observable state}, \quad \kappa\in\{1,\dots,K\}\text{ : latent type}\\[2pt]
&U_{j}(a,z,\kappa,\varepsilon_j)
  = a_j\Big[\alpha_{j}(z_j,\kappa)
        +\sum_{i\neq j}\beta_{ji}(z_j,\kappa)\,a_i\Big] + \varepsilon_j a_j
\end{aligned}

* $\varepsilon_j$ i.i.d. logistic  → logit choice.  
* One shot ⇒ static game of incomplete information.

---

## 2 Conditional Choice Probabilities (CCPs)

\begin{aligned}
p_{j\kappa}(s) = \Pr(a_j=1 \mid z=s,\kappa)
\end{aligned}

Logit best‑response fixed point  

\begin{aligned}
p_{j\kappa}(s)=
\Lambda\!\Bigl(
  \alpha_{j}(z_j,\kappa)
  +\sum_{i\neq j}\beta_{ji}(z_j,\kappa)\,p_{i\kappa}(s)
\Bigr),
\qquad
\Lambda(x)=\frac{1}{1+e^{-x}}
\end{aligned}

`solve_equilibrium()` iterates until convergence.

---

## 3 Mixture of Latent Types

Each market draws a type $\kappa$ with probability $h_s(\kappa)$.

\begin{aligned}
Q_s(a)=\sum_{\kappa=1}^{K}h_s(\kappa)
       \prod_{j=1}^{n}p_{j\kappa}(s)^{a_j}\,[1-p_{j\kappa}(s)]^{1-a_j}
\end{aligned}

`compute_joint_distribution()` implements this formula.

---

## 4 Likelihood & Scores

Observation $(s,a)$ gives log‑likelihood $\ln Q_s(a)$.

Score vectors  

\begin{aligned}
s_{sa}=\frac{\partial Q_s(a)}{\partial\theta}
\end{aligned}

Fisher information / Hessian (per state)  

\begin{aligned}
H_s=\sum_a \frac{s_{sa}s_{sa}^\top}{Q_s(a)}
\end{aligned}

`build_hessian_general()` assembles block‑diagonal $H$.

---

## 5 Equilibrium Constraints

\begin{aligned}
c_{j\kappa}(s)=
\alpha_{j}(z_j,\kappa)+
\sum_{i\neq j}\beta_{ji}(z_j,\kappa)\,p_{i\kappa}(s)
-\operatorname{qlogit}\bigl(p_{j\kappa}(s)\bigr)=0
\end{aligned}

Jacobian entries  

* diag: $-\dfrac{1}{p(1-p)}$  
* off‑diag: $\beta_{ji}$

`build_Dc_hp_general()` returns the Jacobian $D_c$.

---

## 6 What *Identification* Means

Parameters $\theta$ are locally identified  
iff no non‑zero $\delta\theta$ satisfies both

1. $H\,\delta\theta = 0$ (data are flat)  
2. $D_c\,\delta\theta = 0$ (game constraints unchanged)

Stacked matrix  

\begin{aligned}
J = \begin{bmatrix} H \\[4pt] D_c \end{bmatrix}
\end{aligned}

must have **full column rank**.

---

## 7 Rank Diagnostics in the Toolkit

| Matrix | Code name | Reads as |
|--------|-----------|----------|
| $H$        | `rank_hessian`     | Information from data only |
| $H_h$      | `rank_hessian_h`   | Identifies latent‑type probs $h$ |
| $D_c$      | `rank_Dc_hp`       | Restrictions from best‑responses |
| $J=[H;D_c]$| `rank_J`           | Joint identification |

Condition numbers (`cond_H`, `cond_J`, …) warn about near‑singularity (weak ID).

---

## 8 Rules of Thumb

* `rank_J` < full → some parameters not identified → add variation / exclusion.  
* `cond_J` > 10⁸ → very weak identification → expect huge standard errors.  
* One mixture weight is redundant (they sum to 1) → expect one drop in `rank_hessian_h`.

---

## 9 2 × 2 × 2 Example Output

```text
rank_hessian   = 48 
rank_Dc_hp     = 48  
rank_J         = 64  
